# 0. Imports and Setting up Anthropic API Client

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install python-dotenv

import os
import dotenv

dotenv.load_dotenv('/content/drive/MyDrive/.env')

True

In [3]:
# Load Prompts and Problem Description
prompt1_path = '/content/drive/MyDrive/Thesis/Prompts/Prompt1_MathematicalModel.txt'
prompt2_path = '/content/drive/MyDrive/Thesis/Prompts/Prompt2_PyomoCode.txt'
problem_desc_path = '/content/drive/MyDrive/Thesis/ProblemDescriptions/MIP/MIP1.txt'

prompt1_file = open(prompt1_path, "r")
prompt2_file = open(prompt2_path, "r")
problem_desc_file = open(problem_desc_path, "r")

prompt1 = prompt1_file.read()
print("Prompt 1:\n", prompt1)

prompt2 = prompt2_file.read()
print("Prompt 2:\n", prompt2)

problem_desc = problem_desc_file.read()
print("Problem Description:\n", problem_desc)

Prompt 1:
 Please write a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
Prompt 2:
 Please write a python pyomo code for this optimization problem.
Use sample data where needed.
Indicate where you use sample data.
Problem Description:
 You are a city planner, looking to open facilities at some locations. We have a set of customers and a set of possible locations for opening facilities. Each potential location for establishing a facility incurs a fixed annual activation cost, which must be paid if the facility is used, regardless of the service volume it handles. Furthermore, this service volume at each facility is capped at a maximum annual limit. Additionally, there are transportation costs associated with servicing each customer from each facility.
The goal is to minimize the overall costs, which include both the fixed activation costs for any opened facilities and the transportation

In [4]:
!pip install anthropic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 877.7/877.7 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.0 MB/s eta 0:00:00


In [5]:
# Importing Anthropic & Setting Headers
import anthropic

client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key=os.environ.get("ANTHROPIC_API_KEY"),
)

# 1. Prompt 1: Create Mathematical Model

In [6]:
message = client.messages.create(
    model="claude-3-opus-20240229",
    system=prompt1,
    max_tokens=4096,
    messages=[
        {"role": "user", "content": problem_desc}
    ]
)

response_p1 = message.content[0].text

In [7]:
# Print response
print(response_p1)

Great! Let's formulate this problem as a mathematical optimization model. We'll define the necessary sets, parameters, decision variables, objective function, and constraints.

Sets:
- I: Set of customers
- J: Set of potential facility locations

Parameters:
- d_i: Demand of customer i ∈ I
- f_j: Fixed annual activation cost for facility j ∈ J
- c_ij: Transportation cost for servicing customer i ∈ I from facility j ∈ J
- M_j: Maximum annual service volume for facility j ∈ J

Decision Variables:
- x_ij: Fraction of customer i's demand serviced by facility j (continuous variable, 0 ≤ x_ij ≤ 1)
- y_j: Binary variable indicating whether facility j is opened (1) or not (0)

Objective Function:
Minimize total costs = Fixed activation costs + Transportation costs
min ∑_(j∈J) f_j * y_j + ∑_(i∈I) ∑_(j∈J) c_ij * d_i * x_ij

Constraints:
1. Each customer's demand must be fully met:
   ∑_(j∈J) x_ij = 1, ∀i ∈ I

2. Each facility's service volume cannot exceed its maximum annual limit:
   ∑_(i∈I) d_

# 2. Prompt 2: Write Code based on Mathematical Model

In [8]:
message2 = client.messages.create(
    model="claude-3-opus-20240229",
    system=prompt2,
    max_tokens=4069,
    messages=[
        {"role": "user", "content": response_p1}
    ]
)

response_p2 = message2.content[0].text

In [9]:
# Print response
print(response_p2)

Here's the Python code using the Pyomo library to solve the facility location problem:

```python
from pyomo.environ import *

# Sample data
customers = ['C1', 'C2', 'C3', 'C4', 'C5']
facilities = ['F1', 'F2', 'F3']

demand = {'C1': 100, 'C2': 150, 'C3': 200, 'C4': 120, 'C5': 180}  # Sample demand data
fixed_cost = {'F1': 5000, 'F2': 6000, 'F3': 4500}  # Sample fixed activation costs
max_capacity = {'F1': 400, 'F2': 500, 'F3': 450}  # Sample maximum annual service volumes

# Sample transportation costs (c_ij)
trans_cost = {
    ('C1', 'F1'): 2, ('C1', 'F2'): 3, ('C1', 'F3'): 4,
    ('C2', 'F1'): 3, ('C2', 'F2'): 2, ('C2', 'F3'): 3,
    ('C3', 'F1'): 4, ('C3', 'F2'): 3, ('C3', 'F3'): 2,
    ('C4', 'F1'): 3, ('C4', 'F2'): 4, ('C4', 'F3'): 3,
    ('C5', 'F1'): 2, ('C5', 'F2'): 3, ('C5', 'F3'): 4
}

# Create the Pyomo model
model = ConcreteModel()

# Define sets
model.I = Set(initialize=customers)
model.J = Set(initialize=facilities)

# Define parameters
model.d = Param(model.I, initialize

# 4. Input Problem Data and Test Model Code

In [10]:
# Installing pyomo nd solver
!pip install -q pyomo
!apt-get install -y -qq glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 75.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.0 MB/s eta 0:00:00
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 121918 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected pac

In [11]:
from pyomo.environ import *

# Sample data
customers = ['C1', 'C2', 'C3', 'C4', 'C5']
facilities = ['F1', 'F2', 'F3']

demand = {'C1': 80, 'C2': 270, 'C3': 250, 'C4': 160, 'C5': 180}  # Sample demand data
fixed_cost = {'F1': 1000, 'F2': 1000, 'F3': 1000}  # Sample fixed activation costs
max_capacity = {'F1': 500, 'F2': 500, 'F3': 500}  # Sample maximum annual service volumes

# Sample transportation costs (c_ij)
trans_cost = {
    ('C1', 'F1'): 4, ('C1', 'F2'): 6, ('C1', 'F3'): 9,
    ('C2', 'F1'): 5, ('C2', 'F2'): 4, ('C2', 'F3'): 7,
    ('C3', 'F1'): 6, ('C3', 'F2'): 3, ('C3', 'F3'): 4,
    ('C4', 'F1'): 8, ('C4', 'F2'): 5, ('C4', 'F3'): 3,
    ('C5', 'F1'): 10, ('C5', 'F2'): 8, ('C5', 'F3'): 4
}

# Create the Pyomo model
model = ConcreteModel()

# Define sets
model.I = Set(initialize=customers)
model.J = Set(initialize=facilities)

# Define parameters
model.d = Param(model.I, initialize=demand)
model.f = Param(model.J, initialize=fixed_cost)
model.c = Param(model.I, model.J, initialize=trans_cost)
model.M = Param(model.J, initialize=max_capacity)

# Define decision variables
model.x = Var(model.I, model.J, bounds=(0, 1))
model.y = Var(model.J, within=Binary)

# Define objective function
def obj_rule(model):
    return sum(model.f[j] * model.y[j] for j in model.J) + \
           sum(model.c[i, j] * model.d[i] * model.x[i, j] for i in model.I for j in model.J)
model.obj = Objective(rule=obj_rule, sense=minimize)

# Define constraints
def demand_constraint_rule(model, i):
    return sum(model.x[i, j] for j in model.J) == 1
model.demand_constraint = Constraint(model.I, rule=demand_constraint_rule)

def capacity_constraint_rule(model, j):
    return sum(model.d[i] * model.x[i, j] for i in model.I) <= model.M[j] * model.y[j]
model.capacity_constraint = Constraint(model.J, rule=capacity_constraint_rule)

def service_constraint_rule(model, i, j):
    return model.x[i, j] <= model.y[j]
model.service_constraint = Constraint(model.I, model.J, rule=service_constraint_rule)

# Solve the model
solver = SolverFactory('glpk')
results = solver.solve(model)

# Print the results
print("Objective value:", value(model.obj))
print("Facility openings:")
for j in model.J:
    if value(model.y[j]) > 0.5:
        print(f"Facility {j} is opened")
print("Customer assignments:")
for i in model.I:
    for j in model.J:
        if value(model.x[i, j]) > 0.001:
            print(f"Customer {i} is serviced by Facility {j}: {value(model.x[i, j]):.2f}")

Objective value: 5609.999999999998
Facility openings:
Facility F2 is opened
Facility F3 is opened
Customer assignments:
Customer C1 is serviced by Facility F2: 1.00
Customer C2 is serviced by Facility F2: 1.00
Customer C3 is serviced by Facility F2: 0.60
Customer C3 is serviced by Facility F3: 0.40
Customer C4 is serviced by Facility F3: 1.00
Customer C5 is serviced by Facility F3: 1.00


# 5. Correct The Model Code to Test Mathematical Model (if applicable)